# Ejercicio integrador

## El enunciado: WEB SCRAPING

Existen muchas formas de utilizar los buscadores de Internet para encontrar cosas; uno muy
interesante para los entusiastas de las descargas es la de obtener música de forma directa.

Por lo general la música (en forma de archivos) disponible desde un servidor Apache presenta
determinadas palabras en las paginas que al ser indexadas por los buscadores permiten posteriormente formular consultas, combinando estas palabras y lo que efectivamente se esta
buscando de un archivo de música (generalmente el nombre de la canción).

De este modo, un hipotético fanático de Ricardo Arjona podría obtener música simplemente
con:

```bash
intitle:"index of" "Apache Server" "Parent Directory" arjona
```

...o refinando y ajustando un poco mas

```bash
-inurl: (htm|html|php) intitle:"index of" +"last modified" +"parent directory" +description +size +(wma|mp3) arjona
```

...y navegar entre los primeros resultados de la búsqueda para descargar un archivo de audio.

Si dicho fanático quisiera bajar la discografía completa del cantautor de alguna de las urls
encontradas podría utilizar una herramienta como wget para automatizar la descargar de
todos los archivos de audio.

```bash
# !/usr/bin/env bash
wget -r - l1 - np -A . mp3 " $1 "
```

En el ejemplo anterior wget descarga recursivamente (nivel 1) sin subir a la ruta padre todos los mp3s que se obtengan de la url pasada como parámetro.

Considerando que las paginas generadas por Apache son dinámicas, osea que eventualmente
podrian cambiar cuando el administrador del sitio modifique el directorio que contiene a los
archivos y por sobretodo que no a todos tenemos los mismos gustos musicales.

Utilice un lenguaje de programación que, de forma simple, provea funciones para obtener
contenido de internet y codifique una herramienta que descargue música.

## La Solucion: web_scrapper.py

```python
#!/usr/bin/python
# "Copyright (C) 2015 Matías Iglesias"

import re
import sys
import urllib.request
from urllib.error import HTTPError, URLError

ARCHIVO_FUENTES = "fuentes.txt"


def obtener_fuentes():
    # TODO: Manejar el archivo en un bloque with
    f = open(ARCHIVO_FUENTES, "r")
    # TODO: Usar readlines del objeto file
    fuentes = f.read().splitlines(False)
    f.close()
    return fuentes


def scrapp_source(source, search_for, depth=1):
    # print("Abro URL: " + source + " Profundidad: " + str(depth))
    try:
        # TODO: Manejar el request en un bloque with
        f = urllib.request.urlopen(source)
        #print f.read()
        urls = obtener_urls(f.read().decode("utf-8"))
        for url in urls:
            match = re.search(r'\/', url)
            if match and url != "/":
                #print url + " es un directorio"
                if depth > 1:
                    # print("Busco recursivamente en " + source + " subidrectorio " + url)
                    scrapp_source(source + url, search_for, depth - 1)
            else:
                #print url + " es un archivo"
                match = re.search(search_for.lower(), url.lower())
                if match:
                    # print("Bajo " + source + url)
                    # FIXME: file_tmp nunca es cerrado, manejar el request en un bloque with
                    file_tmp = urllib.request.urlopen(source + url)
                    # TODO: Manejar el request en un bloque with
                    local_file = open(url, "wb")
                    local_file.write(file_tmp.read())
                    local_file.close()
        f.close()
        #print urls
    except HTTPError as e:
        print("Ocurrio un error")
        print(e.code)
    except URLError as e:
        print("Ocurrio un error")
        print(e.reason)


def obtener_urls(http):
    # print("HTTP Crudo " + http)
    urls = re.findall(r'href=[\'"]?([^\'" >]+)', http)
    return urls

depth = 1
search_for = None

# TODO: Agregar el bloque if __name__ == '__main__': para trabajar con este 
# script de manera mas limpia ver http://www.artima.com/weblogs/viewpost.jsp?thread=4829

try:
    search_for = sys.argv[1]
    depth = int(sys.argv[2])
except IndexError as ex:
    if search_for is None:
        print("""\
Uso: python web_scrapper.py search [depth]""")
except ValueError as ex:
    print("""\
Uso: python web_scrapper.py search [depth]
depth debe ser un entero""")

if search_for is not None:
    fuentes = obtener_fuentes()
    # print fuentes
    for fuente in fuentes:
        # print("Scrappeo " + fuente)
        scrapp_source(fuente, search_for, depth)
```

### Contenido del archivo ``fuentes.txt``

```text
http://quitmumbling.com/public_html/wp-content/uploads/
http://mhoerner.dyndns.org
http://yottabi.com/musica/
http://yottabi.com/musica/ARJONA/
http://carlosjuez.com/temporal/
http://metropolisfm.com/musica/
http://floatingworldweb.com/MUSIC/@ELECTRONICA/@DJ%20MIXES/
```


## Observaciones

El script presentado por el alumno (Matías) hace lo que tiene que hacer y descarga música de internet veamos ahora como abordo la solucion y como podemos mejorarla (no funcionamente, sino mas bien pythonicamente).

Matías introduce dos librerias o modulos nuevos de python, ``re`` y ``urllib``, antes de realizar la actividad veamos que hacen estos dos modulos.

### [re](https://docs.python.org/3.5/library/re.html)

In [ ]:
import re

# El patron encuentra las ocurrencias "cadena"
re.findall(r'cadena', 'una cadena con espacios')

In [ ]:
# El patron encuentra las ocurrencias de un espacio
re.findall(r'\s', 'una    cadena    con   espacios,  con  muchos  espacios')

In [ ]:
# El patron encuentra las ocurrencias de un espacio o mas
re.findall(r'\s+', 'una    cadena    con   espacios,  con  muchos  espacios')

In [ ]:
# El patron encuentra las ocurrencias de un caracter o mas
re.findall(r'\w+', 'una    cadena    con   espacios,  con  muchos  espacios')

In [ ]:
# El patron encuentra las ocurrencias de un caracter a o s seguido de uno o mas espacios
re.findall(r'[as]\s+', 'una    cadena    con   espacios,  con  muchos  espacios')

In [ ]:
# El patron encuentra las ocurrencias de un caracter a o s seguido de cero o mas espacios
re.findall(r'[as]\s*', 'una    cadena    con   espacios,  con  muchos  espacios')

### [urllib](https://docs.python.org/3.5/library/urllib.html)

In [ ]:
import urllib.request

with urllib.request.urlopen('http://www.google.com') as r:
    r.read()

## Actividades

1. Ejecutar el script
1. Leer e interpetar el código buscar en la ayuda respectiva usando spyder
1. Este script funciona en python 2 pero no en python 3, descomentar todos los print's de debug y arreglar los que correspondan para que funcionen en python 3
1. Alternar la ejecucion del script con comentar y descomentar prints hasta que estemos comodos con la informacion de debug que se nos presenta
1. Implementar los TODO y los FIXME
1. Mejorar el bloque try...except de la funcion scrapp_source, identificar y trabajar sólo sobre los posibles puntos de falla no sobre todo el bloque de la función
1. La funcion scrapp_source, toma dos caminos alternos para cada url dependiendo del tipo de url que corresponda, identificar esos caminos e implementarlos como funciones
1. Modularizar el codigo creando los modulos y/o paquetes correspondientes para separar la logica de manejar archivos (leer y escribir) y la de obtener urls posteriormente crear ``main.py`` y trabajar con los modulos y/o paquetes obtenidos.
